# Deliverable 3: Preprocessing the Data for a Neural Network
increasing epchos to 200 
TRIAL 1

In [123]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [124]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(labels=["EIN", "NAME"], axis = 1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [125]:
# Look at APPLICATION_TYPE value counts for binning
application_value_count = application_df["APPLICATION_TYPE"].value_counts()

In [126]:
# Determine which values to replace if counts are less than 1000?
replace_application = list(application_value_count[application_value_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

In [127]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df["CLASSIFICATION"].value_counts()

In [128]:
# Determine which values to replace if counts are less than 1000?
replace_class = list(classification_value_counts[classification_value_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [129]:
# Generate our categorical variable lists
application_categorical_variables = list(application_df.dtypes[application_df.dtypes == "object"].index)

In [130]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_categorical_variables]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_categorical_variables)


In [131]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(labels=application_categorical_variables, axis=1)

In [132]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values.reshape(-1, 1)
X = application_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [133]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])

hidden_nodes_layer1 = 80

hidden_nodes_layer2 = 30

nn_epochs = tf.keras.models.Sequential()

# First hidden layer
nn_epochs.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_epochs.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_epochs.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_epochs.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 80)                3520      
                                                                 
 dense_40 (Dense)            (None, 30)                2430      
                                                                 
 dense_41 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [134]:
# Compile the model
nn_epochs.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [135]:
#train the model
increase_epochs = nn_epochs.fit(X_train_scaled, y_train, epochs = 200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5671 - accuracy: 0.7230
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7318
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5497 - accuracy: 0.7334
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7325
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5468 - accuracy: 0.7331
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5455 - accuracy: 0.7338
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accuracy: 0.7338
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5451 - accuracy: 0.7345
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5445 - accuracy: 0.7350
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5438 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5343 - accuracy: 0.7405
Epoch 81/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5338 - accuracy: 0.7404
Epoch 82/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5339 - accuracy: 0.7410
Epoch 83/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5341 - accuracy: 0.7399
Epoch 84/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5340 - accuracy: 0.7408
Epoch 85/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5335 - accuracy: 0.7417
Epoch 86/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5339 - accuracy: 0.7404
Epoch 87/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5339 - accuracy: 0.7406
Epoch 88/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5334 - accuracy: 0.7415
Epoch 89/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5336 - accuracy: 

804/804 [==============================] - 2s 2ms/step - loss: 0.5307 - accuracy: 0.7420
Epoch 160/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5313 - accuracy: 0.7421
Epoch 161/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5313 - accuracy: 0.7412
Epoch 162/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5317 - accuracy: 0.7417
Epoch 163/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5314 - accuracy: 0.7415
Epoch 164/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5305 - accuracy: 0.7420
Epoch 165/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5312 - accuracy: 0.7419
Epoch 166/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5313 - accuracy: 0.7418
Epoch 167/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5308 - accuracy: 0.7419
Epoch 168/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5309 - a

In [136]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_epochs.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6268 - accuracy: 0.7249 - 342ms/epoch - 1ms/step
Loss: 0.6267891526222229, Accuracy: 0.7248979806900024


# Changing the relu to tanh to see how that helps
TRIAL 2

In [137]:
nn_tanh = tf.keras.models.Sequential()

# First hidden layer
nn_tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn_tanh.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_tanh.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 80)                3520      
                                                                 
 dense_43 (Dense)            (None, 30)                2430      
                                                                 
 dense_44 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [138]:
# Compile the model
nn_tanh.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [139]:
tanh_fit = nn_tanh.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5706 - accuracy: 0.7208
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7295: 0s -
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5509 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5489 - accuracy: 0.7323
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7323
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7316
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5447 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5438 - accuracy: 0.7335
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5435 - 

804/804 [==============================] - 2s 3ms/step - loss: 0.5307 - accuracy: 0.7417
Epoch 81/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5305 - accuracy: 0.7411
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5304 - accuracy: 0.7420: 0s - loss: 0.5293 - accu
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5305 - accuracy: 0.7420
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5303 - accuracy: 0.7417
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5304 - accuracy: 0.7418
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5304 - accuracy: 0.7421
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5303 - accuracy: 0.7418
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5302 - accuracy: 0.7418
Epoch 89/100
804/804 [==============================] - 1s 2ms/step -

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_tanh.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Reevaluating the code for a potential higher accuracy 
Trial 3

In [141]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [142]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(labels=["EIN", "NAME"], axis = 1)

# Determine which values to replace if counts are less than ...?
replace_application = list(application_value_count[application_value_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_value_counts[classification_value_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [143]:
#Checking out the Income_AMT value count

income_amt_count = application_df["INCOME_AMT"].value_counts()
income_amt_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [144]:
#replacing values to make the columns smaller
replace_income_amt = list(income_amt_count[income_amt_count < 2500].index)

#replace in the DF
for income in replace_income_amt:
    application_df["INCOME_AMT"] = application_df["INCOME_AMT"].replace(income, "Other")
    
#check to make sure the bucketing is successful
application_df["INCOME_AMT"].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             2790
Name: INCOME_AMT, dtype: int64

In [145]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_categorical_variables]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_categorical_variables)


In [146]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(labels=application_categorical_variables, axis=1)

In [147]:
#droping the "special_consideration_n" column
application_df = application_df.drop("SPECIAL_CONSIDERATIONS_N", axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [148]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values.reshape(-1, 1)
X = application_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [149]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])

hidden_nodes_layer1 = 80

hidden_nodes_layer2 = 30

nn_reduced = tf.keras.models.Sequential()

# First hidden layer
nn_reduced.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_reduced.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_reduced.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_reduced.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 80)                3040      
                                                                 
 dense_46 (Dense)            (None, 30)                2430      
                                                                 
 dense_47 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,501
Trainable params: 5,501
Non-trainable params: 0
_________________________________________________________________


In [150]:
# Compile the model
nn_reduced.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [151]:
reduced_model = nn_reduced.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5740 - accuracy: 0.7207
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5596 - accuracy: 0.7270
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5558 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5543 - accuracy: 0.7289
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5539 - accuracy: 0.7287
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7296: 0s -
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5512 - accuracy: 0.7299
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7305
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5500 - accuracy: 0.7314: 0s - los
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss:

Epoch 81/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5381 - accuracy: 0.7380
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7373
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5383 - accuracy: 0.7379: 0s - loss: 0.5382 - accuracy: 0.73
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5384 - accuracy: 0.7382
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5380 - accuracy: 0.7388
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5383 - accuracy: 0.7386
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5378 - accuracy: 0.7379
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5380 - accuracy: 0.7382
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5379 - accuracy: 0.7385
Epoch 90/100
804/804 [========================

In [153]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_reduced.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5624 - accuracy: 0.7247 - 679ms/epoch - 3ms/step
Loss: 0.5623794794082642, Accuracy: 0.7246647477149963


# Reevaluating the code for a potential higher accuracy trial - same as above but different layers and nodes

Trail 4

In [154]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [155]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(labels=["EIN", "NAME"], axis = 1)

# Determine which values to replace if counts are less than ...?
replace_application = list(application_value_count[application_value_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_value_counts[classification_value_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [156]:
#Checking out the Income_AMT value count

income_amt_count = application_df["INCOME_AMT"].value_counts()
income_amt_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [157]:
#replacing values to make the columns smaller
replace_income_amt = list(income_amt_count[income_amt_count < 2500].index)

#replace in the DF
for income in replace_income_amt:
    application_df["INCOME_AMT"] = application_df["INCOME_AMT"].replace(income, "Other")
    
#check to make sure the bucketing is successful
application_df["INCOME_AMT"].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             2790
Name: INCOME_AMT, dtype: int64

In [158]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_categorical_variables]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_categorical_variables)

In [159]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(labels=application_categorical_variables, axis=1)

In [160]:
#droping the "special_consideration_n" column
application_df = application_df.drop("SPECIAL_CONSIDERATIONS_N", axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [161]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values.reshape(-1, 1)
X = application_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [162]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])

hidden_nodes_layer1 = 80

hidden_nodes_layer2 = 50

hidden_nodes_layer3 = 30

nn_optimized = tf.keras.models.Sequential()

# First hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hideen lay
nn_optimized.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_optimized.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 80)                3040      
                                                                 
 dense_49 (Dense)            (None, 50)                4050      
                                                                 
 dense_50 (Dense)            (None, 30)                1530      
                                                                 
 dense_51 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8,651
Trainable params: 8,651
Non-trainable params: 0
_________________________________________________________________


In [163]:
# Compile the model
nn_optimized.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [164]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("optimized_checkpoints/", exist_ok = True)
checkpoint_path = "optimized_checkpoints/weights.{epoch:02d}hdf5"

#create a callback that saved the models weight 
cp_callback = ModelCheckpoint(filepath = checkpoint_path,
                             verbose = 1,
                             save_weights_only = True,
                             save_freq = 4000)
                             

In [165]:
#Train the model
optimized_fit_model = nn_optimized.fit(X_train_scaled, y_train, epochs = 100, callbacks = [cp_callback])

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5706 - accuracy: 0.7200
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5589 - accuracy: 0.7254
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5553 - accuracy: 0.7279
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7287
Epoch 5/100
767/804 [===========================>..] - ETA: 0s - loss: 0.5542 - accuracy: 0.7279
Epoch 00005: saving model to optimized_checkpoints/weights.05hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5526 - accuracy: 0.7288
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7285
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7302
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7299
Epoch 9/100
804/804 [====================

804/804 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7374
Epoch 64/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5382 - accuracy: 0.7370
Epoch 65/100
536/804 [===================>..........] - ETA: 0s - loss: 0.5350 - accuracy: 0.7394
Epoch 00065: saving model to optimized_checkpoints/weights.65hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7368
Epoch 66/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5375 - accuracy: 0.7382
Epoch 67/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5376 - accuracy: 0.7381
Epoch 68/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5380 - accuracy: 0.7373
Epoch 69/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5380 - accuracy: 0.7371
Epoch 70/100
518/804 [==================>...........] - ETA: 0s - loss: 0.5391 - accuracy: 0.7361
Epoch 00070: saving model to optimized_checkpoints

In [166]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5727 - accuracy: 0.7287 - 410ms/epoch - 2ms/step
Loss: 0.572731077671051, Accuracy: 0.7287463545799255


In [ ]:
#Save and export your results to an HDF5 file, and name it AlphabetSoupCharity_Optimization.h5.
nn_optimized.save("AlphabetSoupCharity_Optimization.h5")